### Definition
The user defines the parameters and variants. These can be hierarchical (nested) and swappable.

For each Parameter we have different variants. each has:
1. Variant Name
1. Variant Value

### Instantiation 
In order to instantiate the parameters - we need to make sure that each parameter that is requested (directly or upstream) has a value.
It is possible to select a pre-defined variant for a parameter or to override it with an arbitrary value 

### Lazy Instantiation
1. To have the instantiation in the correct place in the program's execution:
    1. Serialization - Serializing/packaging the app with instructions only
    1. Performance - Instantiating where it makes more sense in the process. Ideally - in the "warm up" phase of the driver
    1. Allowing Overriding

In general, we want to define the possible in the config instead of the DAG to enable configurable swapping without changing code

# Main API

In [2]:
from hypster import lazy, Options, Composer

## Simple data-types

1. construct a dependency graph "compose"
1. given final_vars - create a subgraph that contains all the dependent nodes from final_vars
1. apply the defaults to all the nodes, if there are
1. check for the selections and apply them
1. then apply the overrides
1. if there are any nodes left that need to be selected (Options) and haven't been - error. otherwise - instantiate.


Now, the question is how do we define a graph, subgraph, nodes and dependency.

In [3]:
%%writefile configs.py
from hypster import Options

temperature = Options({"low" : 0.01, "medium" : 0.1, "high" : 1.0}, default="medium")

Writing configs.py


In [4]:
import configs
from hypster import Composer

config = Composer().with_modules(configs).compose()
config.instantiate(final_vars=["temperature"], 
                   selections={"temperature" : "low"}, 
                   overrides={})

{'temperature': 0.01}

In [1]:
import os
os.chdir("..")
os.getcwd()

'/Users/giladrubin/python_workspace/hypster'

In [78]:
%%writefile configs.py
from tests.classes import Thermometer
from hypster import lazy, Options

thermometer = Thermometer(temperature=Options({"low" : 0.01, "medium" : 0.1, "high" : 1.0}, default="medium"), location="home")

Overwriting configs.py


In [41]:
import copy
from typing import Any, Dict, List
from hypster import Options

def instantiate(config_module, vars: List[str], selections: Dict[str, str] = None):
    print(f"DEBUG: Starting instantiation with vars: {vars}, selections: {selections}")
    result = {}
    selections = selections or {}

    for var_name in vars:
        print(f"DEBUG: Processing variable: {var_name}")
        if not hasattr(config_module, var_name):
            raise ValueError(f"Variable '{var_name}' not found in the config module")

        obj = getattr(config_module, var_name)
        print(f"DEBUG: Original object: {obj}")
        new_obj = _recreate_object(obj, selections, prefix=var_name)
        print(f"DEBUG: Recreated object: {new_obj}")
        result[var_name] = new_obj

    return result

def _recreate_object(obj: Any, selections: Dict[str, str], prefix: str = ""):
    print(f"DEBUG: Recreating object with prefix: {prefix}, type: {type(obj)}")
    
    if isinstance(obj, Options):
        print(f"DEBUG: Found Options object: {obj.options}")
        full_path = prefix
        selected_option = selections.get(full_path, obj.default)
        print(f"DEBUG: Selected option: {selected_option}")
        return obj.options[selected_option]

    if isinstance(obj, dict):
        print(f"DEBUG: Processing dictionary: {obj}")
        return {k: _recreate_object(v, selections, f"{prefix}.{k}" if prefix else k) for k, v in obj.items()}

    if isinstance(obj, (list, tuple)):
        print(f"DEBUG: Processing list/tuple: {obj}")
        return type(obj)(_recreate_object(item, selections, f"{prefix}[{i}]") for i, item in enumerate(obj))

    if isinstance(obj, type) or type(obj).__module__ == 'builtins':
        print(f"DEBUG: Skipping built-in or type object: {obj}")
        return obj

    if hasattr(obj, '__dict__'):
        print(f"DEBUG: Processing object with __dict__: {obj.__dict__}")
        new_obj = copy.copy(obj)  # Use shallow copy
        for attr_name, attr_value in obj.__dict__.items():
            new_path = f"{prefix}.{attr_name}" if prefix else attr_name
            print(f"DEBUG: Processing attribute: {new_path}")
            new_value = _recreate_object(attr_value, selections, new_path)
            setattr(new_obj, attr_name, new_value)
        return new_obj

    print(f"DEBUG: Returning object as-is: {obj}")
    return obj

In [42]:
# Usage example
import configs
result = instantiate(configs, ["thermometer"], selections={"thermometer.temperature": "low"})
result["thermometer"].temperature
assert result["thermometer"].temperature == 0.01

DEBUG: Starting instantiation with vars: ['thermometer'], selections: {'thermometer.temperature': 'low'}
DEBUG: Processing variable: thermometer
DEBUG: Original object: <tests.classes.Thermometer object at 0x11238fdf0>
DEBUG: Recreating object with prefix: thermometer, type: <class 'tests.classes.Thermometer'>
DEBUG: Processing object with __dict__: {'temperature': <hypster.variables.Options object at 0x11238c160>, 'location': 'home'}
DEBUG: Processing attribute: thermometer.temperature
DEBUG: Recreating object with prefix: thermometer.temperature, type: <class 'hypster.variables.Options'>
DEBUG: Found Options object: {'low': 0.01, 'medium': 0.1, 'high': 1.0}
DEBUG: Selected option: low
DEBUG: Processing attribute: thermometer.location
DEBUG: Recreating object with prefix: thermometer.location, type: <class 'str'>
DEBUG: Skipping built-in or type object: home
DEBUG: Recreated object: <tests.classes.Thermometer object at 0x117c498a0>


In [4]:
# Usage example
import configs
result = instantiate(configs, vars=["thermometer"], selections={"temperature": "low"})

AssertionError: 

In [8]:
result["thermometer"].temperature

In [ ]:
assert result["thermometer"].temperature == 0.01

In [6]:
import configs
from hypster import Composer

config = Composer().with_modules(configs).compose()
res = config.instantiate(final_vars=["thermometer"], 
                         selections={"thermometer.temperature" : "low"}, 
                         overrides={})

In [7]:
res["thermometer"].temperature

0.01

In [ ]:
%%writefile configs.py
from hypster import Options

temperature = Options({"low" : 0.01, "medium" : 0.1, "high" : 1.0}, default="medium")

# this is only allowed for strings.
llm = Options(["gpt-4o", "gpt-4o-mini", "gpt-4"], default="gpt-4o") 
# will create the same output as:
#llm = Options({"gpt-4o":"gpt-4o", "gpt-4o-mini":"gpt-4o-mini", "gpt-4":"gpt-4"}, default="gpt-4o") 

In [ ]:
import configs
from hypster import Composer

config = Composer().with_modules(configs).compose()
config.instantiate(final_vars=["llm", "temperature"], 
                   selections={"temperature" : "low"}, 
                   overrides={"llm" : "gpt-4-turbo"}) #note that this value is not in the options, but it still works

## Complex data-types (Class, Functions) - Part 1

In [ ]:
from hypster import lazy, Options

#We start with "lazy(X)" to prepare the class and defer instantiation
OpenAiDriver = lazy(OpenAiDriver)
AnthropicDriver = lazy(AnthropicDriver)

llm_driver = Options({"anthropic" : AnthropicDriver(max_tokens=1000),
                      "openai" : OpenAiDriver(500)} #argument name (max_tokens) should be inferred from "lazy()"}
                      ,default="anthropic")

In [ ]:
config = Composer().with_modules(configs).compose()
config.instantiate(final_vars=["llm_driver"], 
                   selections={"llm_driver" : "openai"}, 
                   overrides={"llm_driver.openai.max_tokens" : 200}) #should also work if max_tokens is defined implicitly

## Complex data-types (Class, Functions) - Part 2

In [ ]:
%%writefile configs.py

from hypster import lazy, Options
OpenAiDriver = lazy(OpenAiDriver)
AnthropicDriver = lazy(AnthropicDriver)

llm_driver = OpenAiDriver(model=Options(["gpt-4o", "gpt-4o-mini", "gpt-4"], default="gpt-4o"), 
                          max_tokens=Options({"low" : 200, "medium" : 500, "high" : 1000}, default="medium"))

In [ ]:
config = Composer().with_modules(configs).compose()
config.instantiate(final_vars=["llm_driver"], 
                   selections={}, #takes default values 
                   overrides={"llm_driver.max_tokens" : 300})

## Complex data-types (Class, Functions) - Part 3

In [ ]:
%%writefile configs.py

from hypster import lazy, Options
OpenAiDriver = lazy(OpenAiDriver)
AnthropicDriver = lazy(AnthropicDriver)

model = Options(["gpt-4o", "gpt-4o-mini", "gpt-4"], default="gpt-4o")
llm_driver = OpenAiDriver(model=model, 
                          max_tokens=Options({"low" : 200, "medium" : 500, "high" : 1000}, default="medium"))

In [ ]:
config = Composer().with_modules(x).compose()
#These are equivalent:
selections={"model" : "gpt-4o-mini"} #notice that this refers to the variable "model"
selections={"llm_driver.model" : "gpt-4o-mini"} #this refers to the argument model of the OpenAiDriver class
config.instantiate(final_vars=["llm_driver"], 
                   selections=selections,
                   overrides={"llm_driver.max_tokens" : 300})

## Complex data-types (Class, Functions) - Part 4

In [ ]:
openai_model = Options(["gpt-4o", "gpt-4o-mini", "gpt-4"], default="gpt-4o")
llm_driver = OpenAiDriver(model=openai_model, 
                          max_tokens=Options({"low" : 200, "medium" : 500, "high" : 1000}, default="medium"))

In [ ]:
config = Composer().with_modules(x).compose()
#These are equivalent:
selections={"openai_model" : "gpt-4o-mini"} #notice that this refers to the variable "openai_model"
selections={"llm_driver.model" : "gpt-4o-mini"} #this refers to the argument model of the OpenAiDriver class
config.instantiate(final_vars=["llm_driver"], 
                   selections=selections,
                   overrides={"llm_driver.max_tokens" : 300})

In [ ]:
openai_model = Options(["gpt-4o", "gpt-4o-mini", "gpt-4"], default="gpt-4o")
llm_driver = OpenAiDriver(model=openai_model, 
                          max_tokens=Options({"low" : 200, "medium" : 500, "high" : 1000}, default="medium"))
tabular_driver = OpenAiDriver(model=openai_model, max_tokens=300)

In [ ]:
config = Composer().with_modules(configs).compose()
config.instantiate(final_vars=["llm_driver"], 
                   selections={"openai_model" : "gpt-4o-mini"},
                   overrides={"llm_driver.model" : "gpt-4o"})

In [ ]:
llm_driver = OpenAiDriver(Options(["gpt-4o", "gpt-4o-mini", "gpt-4"], default="gpt-4o"))

In [ ]:
config = Composer().with_modules(configs).compose()
config.instantiate(final_vars=["llm_driver"], 
                   selections={"llm_driver.model" : "gpt-4o-mini"},
                   overrides={"llm_driver.max_tokens" : 300})

In [18]:
# so the hypster variable name is defined either by the variable name itself (openai_model = ...)
# in this case - it'll affect all the classes/functions that use this variable
# if it is "selected" or "overrided" as llm_driver.model - it'll only affect the OpenAiDriver class

In [ ]:
openai_model = Options(["gpt-4o", "gpt-4o-mini", "gpt-4"], default="gpt-4o")
llm_driver = OpenAiDriver(model=openai_model, 
                          max_tokens=Options({"low" : 200, "medium" : 500, "high" : 1000}, default="medium"))
tabular_driver = OpenAiDriver(model=openai_model, max_tokens=1500)

In [ ]:
config = Composer().with_modules(x).compose()
#These are equivalent:
selections={"openai_model" : "gpt-4o-mini"} #will affect llm_driver and tabular_driver
selections={"llm_driver.model" : "gpt-4o-mini"} #will only affect llm_driver and tabular_driver will get the default value
config.instantiate(final_vars=["llm_driver"], 
                   selections=selections,
                   overrides={"llm_driver.max_tokens" : 300})

In [1]:
cache_manager = CacheManager(cache=DiskCache(path=Options(["/tmp", "/var/tmp"], default="/tmp")))

NameError: name 'CacheManager' is not defined

In [ ]:
sql_cache = SqlCache(table="cache")
disk_cache = DiskCache(path=Options(["/tmp", "/var/tmp"], default="/tmp"))
cache_manager = CacheManager(cache=Options([disk_cache, sql_cache])) #this should work and make variant names according to the variable names

In [ ]:
import configs
from hypster import Composer
config = Composer().with_modules(configs).compose()

config.instantiate(final_vars=["cache_manager"], 
                   selections={"cache" : "disk_cache"},
                   overrides={"disk_cache.path" : "new/path"})

In [ ]:
cache__a = "gpt-4o"
cache__b = "gpt-4o-mini"

In [ ]:
cache = Param("cache") 

In [ ]:
#does order matter? do variants need to be defined before their Param?
#what if there are multiple Params with the same name?

In [ ]:
cache = Options([...])
cache = Options({...})

In [ ]:
# eventualy I want to have a structure that has variant_name & variant value
# be careful of circular dependencies

In [ ]:
# eventualy eventually (!) I want to populate all the placeholders in the dependency chain
# either by selecting a variant or by overriding with a value

In [ ]:
cache_manager = CacheManager(cache=cache, when="all"))

In [ ]:
cache_manager = CacheManager(cache=cache, when="all", dest=SqlDest(Credentials(key="..."))))

# Select

1. simple
1. iterable (list, dict, tuple, general iterable?)
1. class
1. function
1. Hypster

In [ ]:
cache.select("a")
# --> "gpt-4o"
# does it update cache inplace? does it keep the rest of the variants? 
# can I do cache.selct("b") afterwards?
cache.select("a")